In [1]:
# 示例代码运行环境
%load_ext watermark
%watermark -p tensorflow,numpy -v -m

CPython 3.6.0
IPython 6.0.0

tensorflow 1.0.1
numpy 1.12.1

compiler   : GCC 4.9.2
system     : Linux
release    : 3.16.0-4-amd64
machine    : x86_64
processor  : 
CPU cores  : 4
interpreter: 64bit


In [2]:
import tensorflow as tf
import numpy as np

In [3]:
from tensorflow.contrib.legacy_seq2seq import basic_rnn_seq2seq, embedding_rnn_seq2seq, sequence_loss
from tensorflow.python.ops import variable_scope


In [4]:
?basic_rnn_seq2seq

In [5]:
vocab = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    '<GO>': 6,
    '<EOS>': 7,
    '<PAD>': 8
}
reverse_vocab = dict([(v, k) for (k, v) in vocab.items()])

In [6]:
tf.reset_default_graph()

In [7]:
# encoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['A', 'B', 'C'])
# decoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['<GO>', 'D', 'E', 'F', 'F', '<EOS>'])

encoder_data = list('ABC')
decoder_data = ['<GO>', 'D', 'E', 'F', 'F', '<EOS>']

encoder_inputs = [tf.constant([vocab[_]]) for _ in encoder_data]
decoder_inputs = [tf.constant([vocab[_]]) for _ in decoder_data]

In [8]:
cell = tf.contrib.rnn.BasicRNNCell(128)
num_encoder_symbols = 9
num_decoder_symbols = 9
embedding_size = 128

outputs, states = embedding_rnn_seq2seq(
    encoder_inputs, decoder_inputs, cell,
    num_encoder_symbols, num_decoder_symbols,
    embedding_size, output_projection=None,
    feed_previous=False)

In [9]:
weights = list(map(lambda _: tf.constant([_], dtype=tf.float32),  [1, 1, 1, 1, 1]))
loss = sequence_loss(outputs[:-1], decoder_inputs[1:], weights)

In [10]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [11]:
with tf.Session() as sess:
    # Training
    sess.run(tf.global_variables_initializer())
    for iteration in range(50):
        sess.run(train_step)
        print(sess.run(loss))
        
    print("---Deocding----")
    
    # Decoding
    with variable_scope.variable_scope(variable_scope.get_variable_scope(), reuse=True):
        decode_decoder_inputs = map(lambda _: tf.constant([vocab[_]]), ['<GO>', 'A', 'A', 'A', 'A', 'A', 'A'])
        outputs, states = embedding_rnn_seq2seq(
            encoder_inputs, decode_decoder_inputs, cell,
            num_encoder_symbols, num_decoder_symbols,
            embedding_size, output_projection=None,
            feed_previous=True)

        for o in outputs:
            m = np.argmax(o.eval(), axis=1)
            print(reverse_vocab[m[0]])

2.1471
1.89836
1.68108
1.49578
1.33911
1.20611
1.09201
0.993014
0.90634
0.82995
0.762314
0.702227
0.648699
0.600891
0.558084
0.519653
0.48506
0.453838
0.425587
0.399958
0.376653
0.355411
0.336008
0.318245
0.301951
0.286975
0.273183
0.260458
0.248697
0.237805
0.227702
0.218314
0.209576
0.201429
0.193822
0.186707
0.180042
0.17379
0.167917
0.162391
0.157186
0.152276
0.147639
0.143254
0.139101
0.135166
0.131431
0.127883
0.124508
0.121296
---Deocding----
D
E
F
F
<EOS>
<EOS>
D


## 用 placeholder 代替 Tensor

placeholder 的类型是 Tensor，因此上述 API 中的 Tensor 都可以用 placeholder 代替，使得训练、测试过程的数据可以变动

In [12]:
type(tf.placeholder(tf.int32, shape=[None]))

tensorflow.python.framework.ops.Tensor

In [13]:
tf.reset_default_graph()

In [14]:
encoder_length = 5
decoder_length = 5

cell = tf.contrib.rnn.BasicRNNCell(128)
num_encoder_symbols = 9
num_decoder_symbols = 9
embedding_size = 128

encoder_placeholders = [tf.placeholder(tf.int32, shape=[None],
                                       name="encoder_%d" % i) for i in range(encoder_length)]
decoder_placeholders = [tf.placeholder(tf.int32, shape=[None],
                                       name="decoder_%d" % i) for i in range(decoder_length)]
target_placeholders = [tf.placeholder(tf.int32, shape=[None],
                                       name="target_%d" % i) for i in range(decoder_length)]
target_weights_placeholders = [tf.placeholder(tf.float32, shape=[None],
                                       name="decoder_weight_%d" % i) for i in range(decoder_length)]
outputs, states = embedding_rnn_seq2seq(
    encoder_placeholders, decoder_placeholders, cell,
    num_encoder_symbols, num_decoder_symbols,
    embedding_size, output_projection=None,
    feed_previous=False)

loss = sequence_loss(outputs, target_placeholders, target_weights_placeholders)
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [15]:
def seq2seq_pad(encoder_inputs, encoder_length, decoder_inputs, decoder_length, vocab, pad_symbol='<PAD>'):
    """
    - encoder_input: A nested list of symbol str for encoding, length: batch_size
    - encoder_length: max length of encoder input
    - decoder_input: A nested list of symbol str for decoding, length: batch_size
    - decoder_length: max length of decoder input
    - vocab: vocabulary index, symbol (str) -> index (int)
    
    Example: 
    ["hello", "world"] -> ["hi", "<EOS>"]
    ["cover", "me"] -> ["roger", "<EOS>"]
    
    seq2seq_pad([['hello', 'world'], ['cover', 'me']], 4, [['hi', '<EOS>'], ['roger', '<EOS>']], 4, vocab)
    
    Assume that index of "<PAD>" is 0

    Output:
    [[0, 0, <index of 'hello'>, <index of 'world'>], [0, 0, <index of 'cover'>, <index of 'me'>]],
    [[<index of 'hi'>, <index of 'EOS'>, 0, 0], [<index of 'roger'>, <index of 'EOS'>, 0, 0]]
    """
    pad_index = vocab[pad_symbol]
    def to_index(inputs, length, pad_from_start=True):
        inputs_to_index = []
        for cur_input in inputs:
            cur_input_to_index = [pad_index] * length
            l = len(cur_input)
            if l < length:
                if pad_from_start:
                    cur_input_to_index[(length - l):] = [vocab[i] for i in cur_input]
                else:
                    cur_input_to_index[:l] = [vocab[i] for i in cur_input]
            else:
                cur_input_to_index = [vocab[i] for i in cur_input[:length]]
            inputs_to_index.append(cur_input_to_index)    
        return inputs_to_index
    return to_index(encoder_inputs, encoder_length, True), to_index(decoder_inputs, decoder_length, False)

In [16]:
seq2seq_pad([['A', 'B'], ['B', 'A']], 5, [['<GO>', 'C', 'D', '<EOS>'], ['<GO>', 'D', 'C', '<EOS>']], 5, vocab)

([[8, 8, 8, 0, 1], [8, 8, 8, 1, 0]], [[6, 2, 3, 7, 8], [6, 3, 2, 7, 8]])

In [17]:
encoder_inputs, decoder_inputs = seq2seq_pad([['A', 'B'], ['B', 'A']],
                                             5, [['<GO>', 'C', 'D', '<EOS>'], ['<GO>', 'D', 'C', '<EOS>']], 5, vocab)

In [18]:
# 对 nested list 进行『转置』，得到 TF seq2seq 需要的输入形状
print(zip(*encoder_inputs))

In [19]:
def left_shift(decoder_inputs, pad_idx):
    # for generating targets
    return [list(input_[1:]) + [pad_idx] for input_ in decoder_inputs]

In [20]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 两对翻译 pairs
    # AB -> CD
    # BA -> EFF
    encoder_inputs, decoder_inputs = seq2seq_pad([['A', 'B'], ['B', 'A']],
                                                 5, [['<GO>', 'C', 'D', '<EOS>'], ['<GO>', 'E', 'F', 'F', '<EOS>']], 8, vocab)
    encoder_inputs = list(zip(*encoder_inputs))
    # 还有一种方案是直接通过 shift decoder_placeholders 来得到 target_placeholders，这样只需要提供 decoder_placeholders 即可
    target_inputs = list(zip(*left_shift(decoder_inputs, vocab['<PAD>'])))
    decoder_inputs = list(zip(*decoder_inputs))
    
    feed_dict = dict()
    # Prepare input data    
    for (i, placeholder) in enumerate(encoder_placeholders):
        # 这里用 placeholder 或者 placeholder.name 都可以
        feed_dict[placeholder.name] = np.asarray(encoder_inputs[i], dtype=int)
    for i in range(len(decoder_placeholders)):
        feed_dict[decoder_placeholders[i].name] = np.asarray(decoder_inputs[i], dtype=int)
        feed_dict[target_placeholders[i].name] = np.asarray(target_inputs[i], dtype=int)        
        # 这里使用 weights 把 <PAD> 的损失屏蔽了
        feed_dict[target_weights_placeholders[i].name] = np.asarray([float(idx != vocab['<PAD>']) for idx in target_inputs[i]],
                                                      dtype=float)

In [23]:
vocab

{'<EOS>': 7,
 '<GO>': 6,
 '<PAD>': 8,
 'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5}

In [24]:
encoder_data

['A', 'B', 'C']

In [25]:
encoder_inputs

[(8, 8), (8, 8), (8, 8), (0, 1), (1, 0)]

In [27]:
encoder_placeholders

[<tf.Tensor 'encoder_0:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'encoder_1:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'encoder_2:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'encoder_3:0' shape=(?,) dtype=int32>,
 <tf.Tensor 'encoder_4:0' shape=(?,) dtype=int32>]

In [28]:
decoder_inputs

[(6, 6), (2, 4), (3, 5), (7, 5), (8, 7), (8, 8), (8, 8), (8, 8)]

In [22]:
feed_dict

{'decoder_0:0': array([6, 6]),
 'decoder_1:0': array([2, 4]),
 'decoder_2:0': array([3, 5]),
 'decoder_3:0': array([7, 5]),
 'decoder_4:0': array([8, 7]),
 'decoder_weight_0:0': array([ 1.,  1.]),
 'decoder_weight_1:0': array([ 1.,  1.]),
 'decoder_weight_2:0': array([ 1.,  1.]),
 'decoder_weight_3:0': array([ 0.,  1.]),
 'decoder_weight_4:0': array([ 0.,  0.]),
 'encoder_0:0': array([8, 8]),
 'encoder_1:0': array([8, 8]),
 'encoder_2:0': array([8, 8]),
 'encoder_3:0': array([0, 1]),
 'encoder_4:0': array([1, 0]),
 'target_0:0': array([2, 4]),
 'target_1:0': array([3, 5]),
 'target_2:0': array([7, 5]),
 'target_3:0': array([8, 7]),
 'target_4:0': array([8, 8])}

In [21]:
    
    # Training
    for iteration in range(50):
        sess.run(train_step, feed_dict)
        print(sess.run(loss, feed_dict)) 

    print("---Deocding----")
    
    # Decoding
    with variable_scope.variable_scope(variable_scope.get_variable_scope(), reuse=True):
        outputs, states = embedding_rnn_seq2seq(
            encoder_placeholders, decoder_placeholders, cell,
            num_encoder_symbols, num_decoder_symbols,
            embedding_size, output_projection=None,
            feed_previous=True)

        for o in outputs:
            # 注意这里也需要提供 feed_dict
            m = np.argmax(o.eval(feed_dict), axis=1)
            print(reverse_vocab[m[0]], reverse_vocab[m[1]])

RuntimeError: Attempted to use a closed Session.